<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-overview" data-toc-modified-id="Project-overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Project overview</a></span></li><li><span><a href="#Scraping-allegro-link-for-offer-references" data-toc-modified-id="Scraping-allegro-link-for-offer-references-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scraping allegro link for offer references</a></span><ul class="toc-item"><li><span><a href="#Understanding-href-structure" data-toc-modified-id="Understanding-href-structure-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Understanding href structure</a></span></li><li><span><a href="#Extracting-all-offers-from-a-page" data-toc-modified-id="Extracting-all-offers-from-a-page-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Extracting all offers from a page</a></span></li><li><span><a href="#Extracting-all-following-pages" data-toc-modified-id="Extracting-all-following-pages-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Extracting all following pages</a></span></li><li><span><a href="#Create-a-list-of-all-offers" data-toc-modified-id="Create-a-list-of-all-offers-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Create a list of all offers</a></span></li></ul></li><li><span><a href="#Extract-data-from-single-offer" data-toc-modified-id="Extract-data-from-single-offer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extract data from single offer</a></span><ul class="toc-item"><li><span><a href="#Extracting-base-information-from-offer-script" data-toc-modified-id="Extracting-base-information-from-offer-script-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Extracting base information from offer script</a></span></li><li><span><a href="#Extracting-offer-details" data-toc-modified-id="Extracting-offer-details-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Extracting offer details</a></span></li><li><span><a href="#Navigating-through-json-dictionary" data-toc-modified-id="Navigating-through-json-dictionary-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Navigating through json dictionary</a></span></li><li><span><a href="#Combining-findings-into-one-function-extracting-offer-data" data-toc-modified-id="Combining-findings-into-one-function-extracting-offer-data-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Combining findings into one function extracting offer data</a></span></li></ul></li><li><span><a href="#Extracting-and-saving-data" data-toc-modified-id="Extracting-and-saving-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extracting and saving data</a></span><ul class="toc-item"><li><span><a href="#Customizing-Timeout-exceptions" data-toc-modified-id="Customizing-Timeout-exceptions-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Customizing Timeout exceptions</a></span></li><li><span><a href="#Creating-a-csv-file-from-scrapped-data" data-toc-modified-id="Creating-a-csv-file-from-scrapped-data-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Creating a csv file from scrapped data</a></span></li><li><span><a href="#Splitting-data-extraction-into-batches" data-toc-modified-id="Splitting-data-extraction-into-batches-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Splitting data extraction into batches</a></span></li></ul></li></ul></div>

In [98]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json
import numpy as np
import timeit 
from datetime import datetime

# Project overview
<a id="project_overview"><a>

Allegro is the leading online marketplace in Poland. The aim of the project is extracting key data from Allegro Automotive, which will be later used in analysis of car offer pricing.

First notebook is focused on scraping Allegro pages with beautiful soup, understanding the html code and extracting desired data. After storing data it will be analyzed in following notebook. 

The aim of this notebook is to serve as a tutorial to data extraction from Allegro and can be easility used for any Automotive Allegro pages, just by changing link below. It can also be adjusted to any product category after edits to key parameter data  described in 3.1

# Scraping allegro link for offer references


Insert link to the first page of Allegro, from which you start your extraction. I would recommend using at least "Samochody Osobowe" filters before. It is best to use some other filters too as Allegro limits shown results to 100 pages, aproximately 6000 offers. 

In [161]:
link="https://allegro.pl/kategoria/osobowe-bmw-4032?bmatch=baseline-al-product-cl-eyesa2-dict43-aut-1-5-0318"

In [162]:
html = urlopen(link)
bs = BeautifulSoup(html, 'lxml')

## Understanding href structure
<a id="href_struct"><a>

In [163]:
for item in bs.findAll("a",href=True)[:10] :
    print(item)
    print("\n\n")

<a class="_w7z6o _fee54_3kwJ4" data-analytics-click-value="logotype" data-analytics-clickable="" href="https://allegro.pl" itemprop="url" title="Allegro.pl - wygodne i bezpieczne zakupy online, największy wybór ofert">
<img alt="Allegro.pl - wygodne i bezpieczne zakupy online, największy wybór ofert" class="_fee54_10SNZ _fee54_3kwJ4 _fee54_3kwJ4" itemprop="logo" src="https://assets.allegrostatic.com/metrum/brand/allegro-347440b030.svg"/>
</a>



<a aria-label="Kliknij, aby przejść do zakładki obserwowane." class="_1h7wt _1dd5x _1bnv4 _274c1_2lQp7 _mitvy _274c1_3TstK" data-analytics-click-value="" data-analytics-clickable="" href="/moje-allegro/zakupy/obserwowane/oferty"><div class="_1fwkl" data-role="watched-offers-icon-wrapper"><svg class="_nem5f"><image class="_274c1_8DNgs" height="100%" width="100%" xlink:href="https://assets.allegrostatic.com/metrum/icon/star-empty-36debc2c31.svg"/></svg> </div></a>



<a class="_1bo4a _sizcr _xu6h2 _w7z6o _uj8z7" data-analytics-clickable="" data-r

It seems all offer hrefs contain "allegro.pl/ogloszenie" in their adress. 

The links to move to next page of search results contain "przejdź do strony" label

In [164]:
bs.findAll("a",href=True)[1].attrs.keys()

dict_keys(['class', 'href', 'aria-label', 'data-analytics-clickable', 'data-analytics-click-value'])

## Extracting all offers from a page


In [166]:
for item in bs.findAll("a", href=re.compile("allegro.pl/ogloszenie"))[:10]:
    print(item.attrs["href"])
    print("\n")

https://allegro.pl/ogloszenie/bmw-4-f36-gran-coupe-430d-m-pakiet-f-vat-48-300-km-8901537808


https://allegro.pl/ogloszenie/bmw-4-f36-gran-coupe-430d-m-pakiet-f-vat-48-300-km-8901537808


https://allegro.pl/ogloszenie/bmw-5gt-520d-m-pakiet-jasna-skora-navi-led-xenon-9125212183


https://allegro.pl/ogloszenie/bmw-5gt-520d-m-pakiet-jasna-skora-navi-led-xenon-9125212183


https://allegro.pl/ogloszenie/bmw-5-3-0i-258km-fabryczny-mpakiet-1-wlasciciel-9134384727


https://allegro.pl/ogloszenie/bmw-5-3-0i-258km-fabryczny-mpakiet-1-wlasciciel-9134384727


https://allegro.pl/ogloszenie/bmw-118d-coupe-mpakiet-pelny-serwis-bezwypadkowa-9134606969


https://allegro.pl/ogloszenie/bmw-118d-coupe-mpakiet-pelny-serwis-bezwypadkowa-9134606969


https://allegro.pl/ogloszenie/bmw-320d-mperformance-serwis-bezwypadek-155tys-km-9145539546


https://allegro.pl/ogloszenie/bmw-320d-mperformance-serwis-bezwypadek-155tys-km-9145539546




In [160]:
def get_page_offers(link):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'lxml')
                    
    offer_list=[]

    for item in bs.findAll("a", href=re.compile("allegro.pl/ogloszenie")):
        offer_list.append(item.attrs["href"])
    offer_list=list(set(offer_list))
    return(offer_list)

## Extracting all following pages


In [105]:
for item in bs.findAll("a", {"aria-label":re.compile("przejdź do strony")}):
    print(item.attrs["href"])
    print("\n\n")

https://allegro.pl/kategoria/osobowe-bmw-4032?p=2



https://allegro.pl/kategoria/osobowe-bmw-4032?p=3



https://allegro.pl/kategoria/osobowe-bmw-4032?p=4



https://allegro.pl/kategoria/osobowe-bmw-4032?p=5



https://allegro.pl/kategoria/osobowe-bmw-4032?p=2



https://allegro.pl/kategoria/osobowe-bmw-4032?p=3



https://allegro.pl/kategoria/osobowe-bmw-4032?p=4



https://allegro.pl/kategoria/osobowe-bmw-4032?p=5





In [106]:
for item in bs.findAll("a", {"aria-label":re.compile("następna")}):
    print(item.attrs["href"])
    print("\n\n")

https://allegro.pl/kategoria/osobowe-bmw-4032?p=2



https://allegro.pl/kategoria/osobowe-bmw-4032?p=2





In [107]:
bs.findAll("a", {"aria-label":re.compile("następna")})[0].attrs["href"]

'https://allegro.pl/kategoria/osobowe-bmw-4032?p=2'

In [108]:
def get_next_page(link):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'lxml')
                    
    next_page=bs.findAll("a", {"aria-label":re.compile("następna")})[0].attrs["href"]

    
    return(next_page)

In [109]:
def get_n_following(link,n):
    page_list=[link]
    link_temp=link
    print("Added page 0 to pages list:\n{}\n".format(link))
    for i in range(0,n):
        try:
            next_page=get_next_page(link_temp)
            print("Added page {} to pages list:\n{}\n".format(i+1,next_page))
            link_temp=next_page
            page_list.append(next_page)
        except:
            break
    return(page_list)

In [156]:
page_list=get_n_following(link,20)

Added page 0 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?bmatch=baseline-al-product-cl-eyesa2-dict43-aut-1-5-0318

Added page 1 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=2

Added page 2 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=3

Added page 3 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=4

Added page 4 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=5

Added page 5 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=6

Added page 6 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=7

Added page 7 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=8

Added page 8 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=9

Added page 9 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=10

Added page 10 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?p=11

Added page 11 to pages list:
https://allegro.pl/kategoria/osobowe-bmw-4032?

In [111]:
len(page_list)

37

## Create a list of all offers 


In [112]:
all_offers=[]
for i in range(0,len(page_list)-1):
    print("Scraping offers from link")
    print(page_list[i])
    all_offers=all_offers+get_page_offers(page_list[i])
    print("Combined number of offers={}\n".format(len(all_offers)))

        

Scraping offers from link
https://allegro.pl/kategoria/osobowe-bmw-4032?bmatch=baseline-al-product-cl-eyesa2-dict43-aut-1-5-0318
Combined number of offers=60

Scraping offers from link
https://allegro.pl/kategoria/osobowe-bmw-4032?p=2
Combined number of offers=120

Scraping offers from link
https://allegro.pl/kategoria/osobowe-bmw-4032?p=3
Combined number of offers=180

Scraping offers from link
https://allegro.pl/kategoria/osobowe-bmw-4032?p=4
Combined number of offers=240

Scraping offers from link
https://allegro.pl/kategoria/osobowe-bmw-4032?p=5
Combined number of offers=300

Scraping offers from link
https://allegro.pl/kategoria/osobowe-bmw-4032?p=6
Combined number of offers=360

Scraping offers from link
https://allegro.pl/kategoria/osobowe-bmw-4032?p=7
Combined number of offers=420

Scraping offers from link
https://allegro.pl/kategoria/osobowe-bmw-4032?p=8
Combined number of offers=480

Scraping offers from link
https://allegro.pl/kategoria/osobowe-bmw-4032?p=9
Combined number 

# Extract data from single offer

In [113]:
offer=all_offers[29]
offer

'https://allegro.pl/ogloszenie/bmw-x3-f25-2016-diesel-xline-190-ps-salon-pl-8960550484'

In [114]:
html = urlopen(offer)
bs = BeautifulSoup(html, 'lxml')

In [115]:
bs.title.string

'BMW X3 F25 2016 diesel xline 190 ps salon PL 8960550484 - Allegro.pl'

In [116]:
#bs.body

In [117]:
bs.title

<title>BMW X3 F25 2016 diesel xline 190 ps salon PL 8960550484 - Allegro.pl</title>

## Extracting base information from offer script

Basic information about offer ID and car model and OEM can be extracted from bs.script.string. From this part of page we only want to extract OEM, model, serie and offer ID

In [118]:
bs.script.string

'dataLayer=[{"idItem":"8960550484","headNavigation":"Samochody|Osobowe|BMW|X3|F25 (2010-)","categoryPath":"149|4029|4032|12440|147328","idCategory":"147328","nameCategory":"F25 (2010-)","sellerName":"uptobuy_pl","active":"yes","sma":true,"price":99000,"pageType":"detail","department":"Motoryzacja","FreeboxOffer":false,"FreeboxDeal":false,"offerName":"BMW X3 F25 2016 diesel xline 190 ps salon PL","sellerId":"43229940","isAuction":false,"isBuyNow":false,"isClassified":true,"isCharity":false,"charityOrganization":null,"fundraisingCampaign":null,"experiments":{"ask-seller-advertisement-captcha":{"variant":"show-captcha"}}}]'

In [119]:
params=re.findall("\[([^]]+)\]",bs.script.string)[0]

In [120]:
params

'{"idItem":"8960550484","headNavigation":"Samochody|Osobowe|BMW|X3|F25 (2010-)","categoryPath":"149|4029|4032|12440|147328","idCategory":"147328","nameCategory":"F25 (2010-)","sellerName":"uptobuy_pl","active":"yes","sma":true,"price":99000,"pageType":"detail","department":"Motoryzacja","FreeboxOffer":false,"FreeboxDeal":false,"offerName":"BMW X3 F25 2016 diesel xline 190 ps salon PL","sellerId":"43229940","isAuction":false,"isBuyNow":false,"isClassified":true,"isCharity":false,"charityOrganization":null,"fundraisingCampaign":null,"experiments":{"ask-seller-advertisement-captcha":{"variant":"show-captcha"}}}'

In [121]:
params_dict=json.loads(params)

In [122]:
offerID=params_dict["idItem"]

In [123]:
params_dict["price"]

99000

In [124]:
OEMdet=params_dict["headNavigation"]

In [125]:
OEMdet

'Samochody|Osobowe|BMW|X3|F25 (2010-)'

In [126]:
OEMdet=re.findall(("(?<=Samochody\|Osobowe)(?s)(.*$)"),OEMdet)[0]

In [127]:
OEMdet

'|BMW|X3|F25 (2010-)'

In [128]:
OEM, model, serie=(OEMdet.split("|"))[1:]

In [129]:
data={}

In [130]:
data["OEM"],data["model"],data["serie"]=(OEMdet.split("|"))[1:]

In [131]:
data["offerID"]=offerID
data["price"]=params_dict["price"]
data["title"]=bs.title.string

In [132]:
data

{'OEM': 'BMW',
 'model': 'X3',
 'serie': 'F25 (2010-)',
 'offerID': '8960550484',
 'price': 99000,
 'title': 'BMW X3 F25 2016 diesel xline 190 ps salon PL 8960550484 - Allegro.pl'}

## Extracting offer details

In [133]:
bs.findAll("script")

[<script nonce="n0t56jkYejk+NHPMqa3iIQ==">dataLayer=[{"idItem":"8960550484","headNavigation":"Samochody|Osobowe|BMW|X3|F25 (2010-)","categoryPath":"149|4029|4032|12440|147328","idCategory":"147328","nameCategory":"F25 (2010-)","sellerName":"uptobuy_pl","active":"yes","sma":true,"price":99000,"pageType":"detail","department":"Motoryzacja","FreeboxOffer":false,"FreeboxDeal":false,"offerName":"BMW X3 F25 2016 diesel xline 190 ps salon PL","sellerId":"43229940","isAuction":false,"isBuyNow":false,"isClassified":true,"isCharity":false,"charityOrganization":null,"fundraisingCampaign":null,"experiments":{"ask-seller-advertisement-captcha":{"variant":"show-captcha"}}}]</script>,
 <script nonce="n0t56jkYejk+NHPMqa3iIQ==">var opbox = opbox || {}; opbox.config = opbox.config || {"baseDomain":"allegro.pl","edgeHost":"https://edge.allegro.pl","cookieMonster":{"host":"https://ngastatic.com/s4c/tracker.js","account":"CM.991213.tz_pl","boxViewDisabled":false,"defaultCustomParams":{"pv":{"itemId":"89605

To find the source of more detailed offer parameters, I looked up mileage ("przebieg") value specific for analyzed offer and found that it appears 10 times within the body. The json data, which looks to be most appropriate for further analysis is stored within data-serialize-box-name="Parameters" attribute 

In [134]:
bs.findAll("script",{'data-serialize-box-name':'Parameters'})[0].string

'{"layout":"TWO_COLUMNS","groups":[{"label":null,"icon":null,"description":null,"singleValueParams":[{"id":"11323","name":"Stan","value":{"id":"11323_2","name":"Używany","url":null}},{"id":null,"name":"Faktura","value":{"id":null,"name":"Wystawiam fakturę VAT","url":null}},{"id":"1","name":"Rok produkcji","value":{"id":null,"name":"2016","url":null}},{"id":"4","name":"Przebieg","value":{"id":null,"name":"123000 km","url":null}},{"id":"5","name":"Pojemność silnika","value":{"id":null,"name":"2000 cm³","url":null}},{"id":"14","name":"Moc","value":{"id":null,"name":"190 KM","url":null}},{"id":"16","name":"Rodzaj paliwa","value":{"id":"16_2","name":"Diesel","url":"https:\\u002F\\u002Fallegro.pl\\u002Fkategoria\\u002Fx3-f25-2010-147328?rodzaj-paliwa=Diesel"}},{"id":"13","name":"Nadwozie","value":{"id":"13_10","name":"SUV","url":"https:\\u002F\\u002Fallegro.pl\\u002Fkategoria\\u002Fx3-f25-2010-147328?nadwozie=SUV"}},{"id":"128789","name":"Liczba miejsc","value":{"id":null,"name":"5","url":nu

The string within Parameter table is a json file, which needs to be unpacked before further analysis

In [135]:
data_dict=bs.findAll("script",{'data-serialize-box-name':'Parameters'})[0].string

In [136]:
data_dict=json.loads(data_dict)

In [137]:
data_dict

{'layout': 'TWO_COLUMNS',
 'groups': [{'label': None,
   'icon': None,
   'description': None,
   'singleValueParams': [{'id': '11323',
     'name': 'Stan',
     'value': {'id': '11323_2', 'name': 'Używany', 'url': None}},
    {'id': None,
     'name': 'Faktura',
     'value': {'id': None, 'name': 'Wystawiam fakturę VAT', 'url': None}},
    {'id': '1',
     'name': 'Rok produkcji',
     'value': {'id': None, 'name': '2016', 'url': None}},
    {'id': '4',
     'name': 'Przebieg',
     'value': {'id': None, 'name': '123000 km', 'url': None}},
    {'id': '5',
     'name': 'Pojemność silnika',
     'value': {'id': None, 'name': '2000 cm³', 'url': None}},
    {'id': '14',
     'name': 'Moc',
     'value': {'id': None, 'name': '190 KM', 'url': None}},
    {'id': '16',
     'name': 'Rodzaj paliwa',
     'value': {'id': '16_2',
      'name': 'Diesel',
      'url': 'https://allegro.pl/kategoria/x3-f25-2010-147328?rodzaj-paliwa=Diesel'}},
    {'id': '13',
     'name': 'Nadwozie',
     'value': {

## Navigating through json dictionary

In [139]:
data_dict.keys()

dict_keys(['layout', 'groups'])

In [140]:
data_dict["groups"][0].keys()

dict_keys(['label', 'icon', 'description', 'singleValueParams', 'multiValueParams'])

The singleValueParams contain key data for further analysis

In [141]:
data_dict["groups"][0]['singleValueParams']

[{'id': '11323',
  'name': 'Stan',
  'value': {'id': '11323_2', 'name': 'Używany', 'url': None}},
 {'id': None,
  'name': 'Faktura',
  'value': {'id': None, 'name': 'Wystawiam fakturę VAT', 'url': None}},
 {'id': '1',
  'name': 'Rok produkcji',
  'value': {'id': None, 'name': '2016', 'url': None}},
 {'id': '4',
  'name': 'Przebieg',
  'value': {'id': None, 'name': '123000 km', 'url': None}},
 {'id': '5',
  'name': 'Pojemność silnika',
  'value': {'id': None, 'name': '2000 cm³', 'url': None}},
 {'id': '14',
  'name': 'Moc',
  'value': {'id': None, 'name': '190 KM', 'url': None}},
 {'id': '16',
  'name': 'Rodzaj paliwa',
  'value': {'id': '16_2',
   'name': 'Diesel',
   'url': 'https://allegro.pl/kategoria/x3-f25-2010-147328?rodzaj-paliwa=Diesel'}},
 {'id': '13',
  'name': 'Nadwozie',
  'value': {'id': '13_10',
   'name': 'SUV',
   'url': 'https://allegro.pl/kategoria/x3-f25-2010-147328?nadwozie=SUV'}},
 {'id': '128789',
  'name': 'Liczba miejsc',
  'value': {'id': None, 'name': '5', 'ur

In [142]:
for i in range(0,len(data_dict["groups"][0]['singleValueParams'])):
    feature=data_dict["groups"][0]['singleValueParams'][i]["name"]
    value=data_dict["groups"][0]['singleValueParams'][i]["value"]["name"]
    data[feature]=value


In [143]:
data

{'OEM': 'BMW',
 'model': 'X3',
 'serie': 'F25 (2010-)',
 'offerID': '8960550484',
 'price': 99000,
 'title': 'BMW X3 F25 2016 diesel xline 190 ps salon PL 8960550484 - Allegro.pl',
 'Stan': 'Używany',
 'Faktura': 'Wystawiam fakturę VAT',
 'Rok produkcji': '2016',
 'Przebieg': '123000 km',
 'Pojemność silnika': '2000 cm³',
 'Moc': '190 KM',
 'Rodzaj paliwa': 'Diesel',
 'Nadwozie': 'SUV',
 'Liczba miejsc': '5',
 'Napęd': '4x4',
 'Kolor': 'Czarny',
 'Lakier': 'Metalik',
 'Kraj pochodzenia': 'Polska',
 'Pochodzenie': 'krajowe',
 'Skrzynia biegów': 'Automatyczna',
 'Klimatyzacja': 'automatyczna dwustrefowa',
 'Oferta dotyczy': 'sprzedaż'}

In [144]:
print(bs.title.string)
print("\n")
for i in range(0,len(data_dict["groups"][0]['singleValueParams'])):
    feature=data_dict["groups"][0]['singleValueParams'][i]["name"]
    value=data_dict["groups"][0]['singleValueParams'][i]["value"]["name"]
    print("{}={}".format(feature, value))

BMW X3 F25 2016 diesel xline 190 ps salon PL 8960550484 - Allegro.pl


Stan=Używany
Faktura=Wystawiam fakturę VAT
Rok produkcji=2016
Przebieg=123000 km
Pojemność silnika=2000 cm³
Moc=190 KM
Rodzaj paliwa=Diesel
Nadwozie=SUV
Liczba miejsc=5
Napęd=4x4
Kolor=Czarny
Lakier=Metalik
Kraj pochodzenia=Polska
Pochodzenie=krajowe
Skrzynia biegów=Automatyczna
Klimatyzacja=automatyczna dwustrefowa
Oferta dotyczy=sprzedaż


## Combining findings into one function extracting offer data

In [145]:
def Extract_data(link):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'lxml')
    
    # Extracting key offer parameters as described in 3.1
    
    key_data=re.findall("\[([^]]+)\]",bs.script.string)[0]
    key_data=json.loads(key_data)
    
    OEMdet=key_data["headNavigation"]  
    OEMdet=re.findall(("(?<=Samochody\|Osobowe)(?s)(.*$)"),OEMdet)[0]
    
    data_dict={}
    
    if len((OEMdet.split("|"))[1:])==3:
    
        data_dict["OEM"],data_dict["model"],data_dict["serie"]=(OEMdet.split("|"))[1:]
    elif len((OEMdet.split("|"))[1:])==2:
        data_dict["OEM"],data_dict["model"]=(OEMdet.split("|"))[1:3]
    else:
        data_dict["OEM"]=(OEMdet.split("|"))[1]
        
    data_dict["offerID"]=key_data["idItem"]
    data_dict["price"]=key_data["price"]
    data_dict["title"]=bs.title.string
    data_dict["download_date"]=str(datetime.now())
    
    #Extracting offer details described in 3.2-3.3
    
    details=bs.findAll("script",{'data-serialize-box-name':'Parameters'})[0].string
    details=json.loads(details)
    offer_details=details["groups"][0]['singleValueParams']
    
    for i in range(0,len(offer_details)):
        feature=offer_details[i]["name"]
        value=offer_details[i]["value"]["name"]
        data_dict[feature]=value
        
    
    

    
    
    return(data_dict)





    
    

In [146]:
offer=all_offers[0]
offer

'https://allegro.pl/ogloszenie/bmw-5-f10-528-i-245-km-9086094556'

Let's investigate how the function works with one of offers

In [147]:
Extract_data(offer)

{'OEM': 'BMW',
 'model': 'Seria 5',
 'serie': 'F10 (2009-2017)',
 'offerID': '9086094556',
 'price': 54000,
 'title': 'BMW 5 (F10) 528 i 245 KM 9086094556 - Allegro.pl',
 'download_date': '2020-04-10 10:40:04.248414',
 'Stan': 'Używany',
 'Faktura': 'Nie wystawiam faktury',
 'Rok produkcji': '2012',
 'Przebieg': '183000 km',
 'Pojemność silnika': '1997 cm³',
 'Moc': '245 KM',
 'Rodzaj paliwa': 'Benzyna',
 'Nadwozie': 'Sedan',
 'Liczba drzwi': '4/5',
 'Liczba miejsc': '5',
 'Napęd': 'Na tylne koła',
 'Kolor': 'Czarny',
 'Lakier': 'Metalik',
 'Kierownica po prawej (Anglik)': 'Nie',
 'Kraj pochodzenia': 'Stany Zjednoczone',
 'Informacje dodatkowe': 'Pierwszy właściciel',
 'Pochodzenie': 'import',
 'Skrzynia biegów': 'Automatyczna',
 'Klimatyzacja': 'automatyczna dwustrefowa',
 'Oferta dotyczy': 'sprzedaż',
 'Stan blacharsko-lakierniczy': 'naprawy blacharsko-lakiernicze',
 'Numer VIN': 'WBAXG5C54CDX01911'}

# Extracting and saving data 

## Customizing Timeout exceptions

Defining custom Timeout exceptions is crucial to avoid the crawler stopping at one page for too long. It can be done using a simple decorator. 

In [148]:
from contextlib import contextmanager
import threading
import _thread

class TimeoutException(Exception):
    def __init__(self, msg=''):
        self.msg = msg

@contextmanager
def time_limit(seconds, msg=''):
    timer = threading.Timer(seconds, lambda: _thread.interrupt_main())
    timer.start()
    try:
        yield
    except KeyboardInterrupt:
        pass 
    finally:
        timer.cancel()


## Creating a csv file from scrapped data

In [149]:
#This function allows to merge multiple offers data into a list of data dictionaries in json format 
#and then transform them into o DataFrame

def Offer_database(offers_list,time,directory):
    data=[]
    failed_extractions=0
    for i in range(0, len(offers_list)):
        
        #Each loops tries to extract data from one offer until "time" amount of seconds is exceeded. Exceeding time limit 
        #passes the loop to the next offer
        
        with time_limit(time, 'sleep'):
            try:
                offer=offers_list[i]
                if np.mod(i,100)==0:
                    print("Extraction: {}/{} at:{}".format(i+1,len(offers_list),str(datetime.now())[:-7] ))
                offer_data=Extract_data(offer)
                data.append(offer_data)
               
            
                
            

            except:
                print("Extraction: {}/{}  !!FAILED!!".format(i+1,len(offers_list)))
                failed_extractions=failed_extractions+1
                pass
            
    #After trying to scrap data from all offers, the summary of results is printed.
    
    print("\n------------------------------------------\n")
    print("Sucessfully Extracted: {}/{} offers \nErrors:{}".format((i-failed_extractions+1),i+1,failed_extractions))
    
    #Then data is converted from json to DataFrame and saved as a csv file in given directory
    
    data_json=json.dumps(data)
    df=pd.read_json(data_json, orient='records')
    
    timestring=str(datetime.utcnow().time())[:8]
    timestring=timestring.replace(":","-")
    timestring
    print("Saving {} offers at{}".format(i-failed_extractions+1,directory+"/"+timestring+".csv"))
   
    df.to_csv(directory+"/"+timestring+".csv")  

        

## Splitting data extraction into batches

After initial attempt to use function from 4.2. on a list of 2k+ offers, I realized that this sort of extraction is not the most effective due to high memory and disc usage. I decided to divide the extraction into batches of n offers.

In [150]:
# Create a function to split offers into a list of lists not exceeding batch_size
def split_offers(offer_list, batch_size):
    offers_split=[offer_list[x:x+batch_size] for x in range(0, len(offer_list), batch_size)]
    return(offers_split)

In [151]:
#Loop over Offer_database function for each batch of offers and save them in one folder with today's date
def Offer_database_batches(offers_list, time, batch_size,folder_name):
    import os
    todayDate = str(datetime.utcnow().date())
    directory = './'+folder_name+"_" + todayDate
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    
    offers_split=split_offers(offers_list, batch_size)
    for i in range(0, len(offers_split)):
        print("Starting Batch no.{}/{}\n".format(i+1,len(offers_split)))
        offers=offers_split[i]
        Offer_database(offers,time,directory)
        print("\n------------------------------------------\nFinished Batch no.{}/{}\n\n".format(i+1,len(offers_split)))

In [152]:
len(all_offers)

2160

We have 2160 offers extracted from our initial link. Let's extract them in batches of 500 and save as 5 csv files, which we will merge later. The timeout limit has been set to 20s.

In [153]:
Offer_database_batches(all_offers,20,500,"BMW_Database")

Starting Batch no.1/5

Extraction: 1/500 at:2020-04-10 10:40:19
Extraction: 101/500 at:2020-04-10 10:41:18
Extraction: 201/500 at:2020-04-10 10:42:17
Extraction: 226/500  !!FAILED!!

Extraction: 301/500 at:2020-04-10 10:43:17
Extraction: 401/500 at:2020-04-10 10:44:16

------------------------------------------

Sucessfully Extracted: 499/500 offers 
Errors:1
Saving 499 offers at./BMW_Database_2020-04-10/08-45-16.csv

------------------------------------------
Finished Batch no.1/5


Starting Batch no.2/5

Extraction: 1/500 at:2020-04-10 10:45:16
Extraction: 101/500 at:2020-04-10 10:46:29
Extraction: 201/500 at:2020-04-10 10:47:29
Extraction: 301/500 at:2020-04-10 10:48:29
Extraction: 401/500 at:2020-04-10 10:49:28
Extraction: 426/500  !!FAILED!!


------------------------------------------

Sucessfully Extracted: 499/500 offers 
Errors:1
Saving 499 offers at./BMW_Database_2020-04-10/08-50-27.csv

------------------------------------------
Finished Batch no.2/5


Starting Batch no.3/5
